In [12]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Nov 14 10:48:00 2024

@author: odarroyo
"""

from openseespy.opensees import *
import matplotlib.pyplot as plt
import opseestools.analisis as an
import opseestools.utilidades as ut
import opsvis as opsv
import numpy as np
import opstool as opst
import opstool.vis.pyvista as opsvis
import opstool.vis.plotly as opsvis2


In [13]:
#%%

wipe()
model('basic','-ndm',2,'-ndf',3)

#%% Nodos y apoyos

coordx = [0, 6.0, 10.0]
coordy = [0, 4.0, 7.0]
ut.creategrid(coordx,coordy)
fixY(0,1,1,1) 

mass(1001,1,1,0) 
mass(1002,1,1,0)
mass(2001,1,1,0) 
mass(2002,1,1,0)
mass(3001,1,1,0) 
mass(3002,1,1,0)

#%% Materiales

fc = 28 # en MPa
fy = 420 # en MPa
tag_noconf, tag_conf, tag_acero = ut.col_materials(fc,fy)

#%% Secciones

Bcol = 0.3 # base de la columna
Hcol = 0.3 # altura de la columna
Bvig = 0.3 # base de la viga
Hvig = 0.4 # altura de la viga
c = 0.05  # recubrimiento de las secciones
nFibCover, nFibCore, nFibZcore  = 8, 16, 10 # numero de fibras en zona no confinada, en Z (x) en zona confinada y Y (y) zona confinada
As4 = 0.000127 # area barra #4
As5 = 0.0002 # area barra #5

col30x30 = 101 # tag de la columna
vig30x40 = 201 # tag de la viga

ut.create_rect_RC_section(col30x30, Hcol, Bcol, c, tag_conf, tag_noconf, tag_acero , 4, As5, 4, As5, 4, As5)
ut.create_rect_RC_section(vig30x40, Hvig, Bvig, c, tag_conf, tag_noconf, tag_acero , 3, As4, 4, As4)


#%% Elementos

tagcols,tagbeams = ut.create_elements(coordx,coordy,col30x30,vig30x40) 

#%% Cargas

ut.load_beams(-30,-20,tagbeams)

# Plotting using the opstool library

opst.post.save_model_data(odb_tag=1)
opsvis2.set_plot_props(point_size=0, line_width=3)
fig = opsvis2.plot_model(odb_tag=1, show_outline=True)
fig.update_layout(width=800,
                  height=800)
fig.show()


OPSTOOL ::  Model data has been saved to _OPSTOOL_ODB/ModelData-1.nc!

OPSTOOL ::  Loading model data from _OPSTOOL_ODB/ModelData-1.nc ...

In [14]:

#%% Analisis gravedad y ver modelo

an.gravedad()
loadConst('-time',0.0)
# opsvis.set_plot_props(line_width=4, point_size=2, notebook=True)
# opsvis.plot_model()


#%% Analisis pushover

tiempo, techo = an.dinamicoIDA2DB('GM01.txt',0.01,3000,0.01,9.81,0.025,getNodeTags()[-1],1)


OPSTOOL ::  All responses data with odb_tag = 1000 saved in _OPSTOOL_ODB/RespStepData-1000.nc!

In [ ]:

opsvis.set_plot_props(font_size=8,
                      title_font_size=10,
                      line_width=5,
                      point_size=3)
opsvis.plot_nodal_responses_animation(
    odb_tag=1000, # by default IDA2DB uses an odb_tag of 1000.
    resp_type="disp",
    resp_dof=["UX", "UY"],
    framerate=30,
    savefig="dinamico_anim.mp4",
    scale=3.0,
).close()



OPSTOOL ::  Loading response data from _OPSTOOL_ODB/RespStepData-1000.nc ...

Animation saved to .dinamico_anim.mp4!


In [18]:
from IPython.display import Video

Video(".dinamico_anim.mp4", embed=True, width=800, height=600)